## PART 6 - Concatenation des CV

In [61]:
# Chargement des librairies nécessaires
import pandas as pd
import spacy

# Chargement de la dataset
dataset = pd.read_csv("dataset_text_brut_200.csv", delimiter = ";")

In [62]:
# Instanciation de la bibliothèque fr_core_news_lg de Spacy
nlp = spacy.load("fr_core_news_lg")

In [63]:
# Tokenization des textes de CV
dataset["Tokenization_preparation"] = dataset["CV_text_brut"].apply(lambda x : str(x).replace("\n", " "))
dataset["Tokenization"] = dataset["Tokenization_preparation"].apply(lambda x : nlp(x))
dataset["Number_of_tokens"] = dataset["Tokenization"].apply(lambda x : len(x))

In [64]:
dataset.head()

,CV_number,CV_text_brut,Tokenization_preparation,Tokenization,Number_of_tokens
0,CV_1,SELMA LAFKIR CORDE 80 CODEUSE ENTHOUSIASTE PRO...,SELMA LAFKIR CORDE 80 CODEUSE ENTHOUSIASTE PRO...,"(SELMA, LAFKIR, CORDE, 80, CODEUSE, ENTHOUSIAS...",214
1,CV_2,Ananya Singh \nProfesseure de Lycée\nPROFIL PE...,Ananya Singh Professeure de Lycée PROFIL PERS...,"(Ananya, Singh, , Professeure, de, Lycée, PRO...",291
2,CV_3,"Samira Hadid \n16 place Urbain II, 63000 Clerm...","Samira Hadid 16 place Urbain II, 63000 Clermo...","(Samira, Hadid, , 16, place, Urbain, II, ,, 6...",315
3,CV_4,Carine Maurice \nIngénieure logiciel avec 10 a...,Carine Maurice Ingénieure logiciel avec 10 an...,"(Carine, Maurice, , Ingénieure, logiciel, ave...",292
4,CV_5,MARCELLE ANDRÉ EXPÉRIENCE PROFESSIONNELLE \nAS...,MARCELLE ANDRÉ EXPÉRIENCE PROFESSIONNELLE ASS...,"(MARCELLE, ANDRÉ, EXPÉRIENCE, PROFESSIONNELLE,...",284


In [65]:
dataset.shape

(200, 5)

In [67]:
dataset_concat = pd.DataFrame()

for cv in range(len(dataset)): 
    ## Création d'une dataframe pour accueillir les features des CV
    df = pd.DataFrame(columns = ['CV_Sentences'])
    df.head()
    
    ## Extraction du texte du CV pour appliquer la sentencisation des CV
    index = dataset.columns.get_loc("Tokenization_preparation")

    doc = dataset.iloc[cv,index]                                                             
    doc = nlp(doc)  
    
    ## Création d'une liste de phrases qui va accueillir le texte découpé en phrases grâce à la fonction doc.sents
    list_sentence = []
    for sent in doc.sents :
        list_sentence.append(sent)

    ## Affectation des phrases dans la colonne CV_Sentences et suppression des parenthèses
    df['CV_Sentences'] = list_sentence
    df['Sentences_CV_clean'] = df['CV_Sentences'].apply(lambda x : str(x).replace("(","").replace(")",""))## Création de la colonne CV_number pour identifier les phrases de chaque CV
    index2 = dataset.columns.get_loc("CV_number")
    df['CV_Number'] = dataset.iloc[cv,index2]                                              

    ## Création d'une colonne qui compte le nombre de phrase par CV
    df['Sentence_line'] = df.index     

    ## Création d'une colonne qui compte le nombre de tokens dans la phrase
    df['Nb_tokens'] = df['CV_Sentences'].apply(lambda x : len(x))

    ## Création d'une colonne pour connaitre la position en pourcentage de la phrase par rapport au texte entier
    df['%texte_lu'] = 0
    token_count = 0
    for i in range(len(df)):
        token_count += df['Nb_tokens'][i]
        pourcentage_debut = ((token_count / df['Nb_tokens'].sum()) * 100)
        df['%texte_lu'][i] = (f"{pourcentage_debut:.2f}")

    ## Création pour connaître le % de texte lu à partir de la fin de ligne
    df["%texte_lu_fin_ligne"] = 0
    nb_tokens_total = df["Nb_tokens"].sum()
    sum_token = 0

    for k in range(len(df)):
        sum_token += df['Nb_tokens'][k]
        resultat = ((nb_tokens_total - sum_token) / nb_tokens_total) * 100
        df["%texte_lu_fin_ligne"][k] = (f"{resultat:.2f}")

    ## Création d'une colonne pour savoir si le mot est alphanumerique ou pas
    df["Is_alpha"] = 0
    index3 = df.columns.get_loc("Sentences_CV_clean")
    for l in range(len(df)):
        doc = df.iloc[l,index3]
        doc = nlp(doc)
        list_token_is_alpha = []
        for token in doc:
            list_token_is_alpha.append(token.is_alpha)
        df["Is_alpha"][l] = list_token_is_alpha

    ## Création de la colonne Grammaire pour connaitre le type de mots de la phrase
    df["Grammar"] = 0
    index = df.columns.get_loc("Sentences_CV_clean")
    for m in range(len(df)):
        doc = df.iloc[m,index]
        doc = nlp(doc)
        list_token_pos = []
        for token in doc:
            list_token_pos.append(token.pos_)
            df["Grammar"][m] = list_token_pos

    ## Création de la colonne Label qui sera notre target. Soit 1 il faut supprimer soit 0 on doit garder
    df['Label'] = 0

    ## Création de la dataset avec les CV concatenés
    dataset_concat = dataset_concat.append(df)

## Création d'un fichier Excel avec toutes les phrases de tous les CV
PATH = "/Users/kinn/Desktop/Projet_BlindCV/dataset_CV_200.csv"
dataset_concat.to_csv(path_or_buf = PATH, index = False, sep = ";")

<ipython-input-67-5ba8c267383a>:37: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['%texte_lu'][i] = (f"{pourcentage_debut:.2f}")
<ipython-input-67-5ba8c267383a>:47: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df["%texte_lu_fin_ligne"][k] = (f"{resultat:.2f}")
<ipython-input-67-5ba8c267383a>:58: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df["Is_alpha"][l] = list_token_is_alpha
<ipython-input-67-5ba8c267383a>:69: SettingWithCopyW